# Forecast de Demanda por Plato (Horizonte Próxima Semana)

Este notebook se enfoca **solo** en la etapa de _modelo predictivo_:

- Parte de un dataset histórico diario por plato (`dataset_forecast_diario.csv`).
- Reconstruye las **mismas features** usadas para entrenar los modelos:
  - lags (`lag_1`, `lag_7`, `lag_14`),
  - promedios móviles y desvíos (`rolling_mean_7`, `rolling_mean_14`, etc.),
  - variables de calendario (día del año, semana, codificación cíclica, etc.).
- Para cada `id_plato`:
  - Carga su modelo correspondiente desde la carpeta `models/`.
  - Genera un **pronóstico secuencial** para los próximos `HORIZON_DAYS` días
    (por defecto, la próxima semana).
- Produce como salida el archivo `forecast_plato.csv` con columnas:

```text
fecha | id_plato | demanda_predicha
```

Este archivo se utilizará luego en el notebook de **planeación de compras y simulación de inventario**.



## 1. Imports, rutas y parámetros globales


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

import xgboost as xgb
# import joblib        # si quieres usar Random Forest u otros modelos sklearn
# from tensorflow.keras.models import load_model  # si usas LSTM, descomenta

# Rutas base (ajusta BASE_DIR según tu estructura)
BASE_DIR = Path("../..").resolve()
DATA_PROC = BASE_DIR  / "data" / "processed"
MODELS_DIR = BASE_DIR  / "models" # FIX: Adjusted path for models

print("BASE_DIR:", BASE_DIR)
print("DATA_PROC:", DATA_PROC)
print("MODELS_DIR:", MODELS_DIR)

# Horizonte de forecast (por ejemplo, la próxima semana)
HORIZON_DAYS = 7

BASE_DIR: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion
DATA_PROC: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\data\processed
MODELS_DIR: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\models


## 2. Carga del dataset histórico `dataset_forecast_diario.csv`


In [7]:
# Dataset usado para entrenar los modelos (histórico diario por plato)
CANDIDATE_PATHS = [
    DATA_PROC / "dataset_forecast_diario.csv",
    Path("/content/-1INF46-Plan_Compras_Produccion/data/processed/dataset_forecast_diario.csv"),
    Path("/workspace/-1INF46-Plan_Compras_Produccion/data/processed/dataset_forecast_diario.csv"),
]

DATA_PATH = None
for p in CANDIDATE_PATHS:
    if p.exists():
        DATA_PATH = p
        break

print("DATA_PATH encontrado:", DATA_PATH)
assert DATA_PATH is not None, "No se encontró dataset_forecast_diario.csv. Ajusta las rutas si es necesario."

df_hist = pd.read_csv(DATA_PATH)

print("Shape dataset:", df_hist.shape)
df_hist.head()



DATA_PATH encontrado: C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\data\processed\dataset_forecast_diario.csv
Shape dataset: (21530, 46)


,fecha,plato,cantidad,monto_total,anio,mes,dia,dow,fin_de_semana,feriado,...,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_28,rolling_std_28,dow_sin,dow_cos,mes_sin,mes_cos
0,2021-01-15,1,13.0,338.0,2021,1,15,4,0,0,...,19.000000,4.123106,19.642857,5.415180,NaN,NaN,-0.433884,-0.900969,0.0,1.0
1,2021-01-16,1,27.0,702.0,2021,1,16,5,1,0,...,18.714286,4.498677,18.428571,4.783212,NaN,NaN,-0.974928,-0.222521,0.0,1.0
2,2021-01-17,1,19.0,494.0,2021,1,17,6,1,0,...,20.714286,4.644505,19.071429,5.298248,NaN,NaN,-0.781831,0.623490,0.0,1.0
3,2021-01-18,1,15.0,390.0,2021,1,18,0,0,0,...,20.571429,4.685337,18.642857,5.017004,NaN,NaN,0.000000,1.000000,0.0,1.0
4,2021-01-19,1,17.0,442.0,2021,1,19,1,0,0,...,20.285714,4.990467,18.285714,5.090392,NaN,NaN,0.781831,0.623490,0.0,1.0


## 3. Detección de columnas clave (fecha, target, plato)


In [8]:
# Listas de candidatos para flexibilizar nombres de columnas
DATE_COL_CANDS = ["fecha","ds","date","FECHA"]
TARGET_CANDS   = ["cantidad","ventas","ventas_total","venta_total","y","target","ventas_real"]
PLATO_CANDS    = ["id_plato","plato","plato_id","producto","categoria"]

def pick_col(cands, cols):
    cols_lower = [c.lower() for c in cols]
    colmap = {c.lower(): c for c in cols}
    for cand in cands:
        if cand.lower() in cols_lower:
            return colmap[cand.lower()]
    return None

DATE_COL   = pick_col(DATE_COL_CANDS, df_hist.columns)
TARGET_COL = pick_col(TARGET_CANDS, df_hist.columns)
PLATO_COL  = pick_col(PLATO_CANDS, df_hist.columns)

print("DATE_COL  :", DATE_COL)
print("TARGET_COL:", TARGET_COL)
print("PLATO_COL :", PLATO_COL)

assert DATE_COL and TARGET_COL and PLATO_COL, "No se detectaron correctamente las columnas clave"

# Normalizar tipos
df_hist[DATE_COL] = pd.to_datetime(df_hist[DATE_COL], errors="coerce")
df_hist = df_hist.sort_values([PLATO_COL, DATE_COL]).reset_index(drop=True)

df_hist[[DATE_COL, PLATO_COL, TARGET_COL]].head()



DATE_COL  : fecha
TARGET_COL: cantidad
PLATO_COL : plato


,fecha,plato,cantidad
0,2021-01-15,1,13.0
1,2021-01-16,1,27.0
2,2021-01-17,1,19.0
3,2021-01-18,1,15.0
4,2021-01-19,1,17.0


## 4. Construcción de features (calendario + lags / rolling)


In [9]:
def add_calendar_features(df, date_col):
    """Agrega variables de calendario usadas como regresores."""
    d = df.copy()
    d["year"] = d[date_col].dt.year
    d["weekofyear"] = d[date_col].dt.isocalendar().week.astype(int)
    d["dayofyear"] = d[date_col].dt.dayofyear
    d["is_month_end"] = d[date_col].dt.is_month_end.astype(int)
    d["is_month_start"] = d[date_col].dt.is_month_start.astype(int)
    d["is_quarter_end"] = d[date_col].dt.is_quarter_end.astype(int)
    d["is_quarter_start"] = d[date_col].dt.is_quarter_start.astype(int)

    dow = d[date_col].dt.dayofweek.values
    d["dow"] = dow
    d["dow_sin"] = np.sin(2*np.pi*dow/7.0)
    d["dow_cos"] = np.cos(2*np.pi*dow/7.0)

    mm = d[date_col].dt.month.values
    d["mes"] = mm
    d["mes_sin"] = np.sin(2*np.pi*mm/12.0)
    d["mes_cos"] = np.cos(2*np.pi*mm/12.0)
    return d

def add_lags_rollings(g, target_col, lags=[1,7,14,28], wins=[7,14,28]):
    """Calcula lags y ventanas móviles para una serie por plato."""
    g = g.sort_values(DATE_COL).copy()
    for L in lags:
        g[f"lag_{L}"] = g[target_col].shift(L)
    for W in wins:
        g[f"rolling_mean_{W}"] = g[target_col].shift(1).rolling(W).mean()
        g[f"rolling_std_{W}"]  = g[target_col].shift(1).rolling(W).std()
    return g

# Aplicamos features de calendario al dataset completo
df_hist = add_calendar_features(df_hist, DATE_COL)

# Calculamos lags/rolling por plato
df_hist = df_hist.groupby(PLATO_COL, group_keys=False).apply(
    lambda g: add_lags_rollings(g, TARGET_COL)
)

df_hist.head()


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\783715008.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_hist = df_hist.groupby(PLATO_COL, group_keys=False).apply(


,fecha,plato,cantidad,monto_total,anio,mes,dia,dow,fin_de_semana,feriado,...,dow_cos,mes_sin,mes_cos,year,weekofyear,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start
0,2021-01-15,1,13.0,338.0,2021,1,15,4,0,0,...,-0.900969,0.5,0.866025,2021,2,15,0,0,0,0
1,2021-01-16,1,27.0,702.0,2021,1,16,5,1,0,...,-0.222521,0.5,0.866025,2021,2,16,0,0,0,0
2,2021-01-17,1,19.0,494.0,2021,1,17,6,1,0,...,0.623490,0.5,0.866025,2021,2,17,0,0,0,0
3,2021-01-18,1,15.0,390.0,2021,1,18,0,0,0,...,1.000000,0.5,0.866025,2021,3,18,0,0,0,0
4,2021-01-19,1,17.0,442.0,2021,1,19,1,0,0,...,0.623490,0.5,0.866025,2021,3,19,0,0,0,0


## 5. Definición del vector de entrada al modelo


In [10]:
# Vector de features que se usarán como X en los modelos
FEATURE_COLS = [
    "monto_total", "anio", "mes", "dia", "dow", "fin_de_semana", "feriado",
    "lag_1","lag_7","lag_14","lag_28",
    "rolling_mean_7","rolling_mean_14","rolling_mean_28",
    "rolling_std_7","rolling_std_14","rolling_std_28",
    "dow_sin","dow_cos","mes_sin","mes_cos"
]

missing_feats = [c for c in FEATURE_COLS if c not in df_hist.columns]
print("Features faltantes (si hay):", missing_feats)

# Filtramos el dataset a filas donde todos los features estén disponibles
df_model = df_hist.dropna(subset=FEATURE_COLS + [TARGET_COL]).copy()

print("Shape modelable:", df_model.shape)
df_model[[DATE_COL, PLATO_COL, TARGET_COL] + FEATURE_COLS].head(3)


Features faltantes (si hay): []
Shape modelable: (21194, 53)


,fecha,plato,cantidad,monto_total,anio,mes,dia,dow,fin_de_semana,feriado,...,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_std_7,rolling_std_14,rolling_std_28,dow_sin,dow_cos,mes_sin,mes_cos
28,2021-02-12,1,18.0,468.0,2021,2,12,4,0,0,...,18.571429,19.214286,18.535714,4.755949,4.440696,4.203016,-0.433884,-0.900969,0.866025,0.5
29,2021-02-13,1,16.0,416.0,2021,2,13,5,1,0,...,19.571429,19.285714,18.714286,3.457222,4.410278,4.062996,-0.974928,-0.222521,0.866025,0.5
30,2021-02-14,1,27.0,702.0,2021,2,14,6,1,0,...,19.000000,18.714286,18.321429,3.696846,4.268463,3.752072,-0.781831,0.623490,0.866025,0.5


## 6. Carga del modelo predictivo por plato

Aquí debes adaptar la ruta y tipo de modelo según cómo los guardaste.


In [11]:
def load_model_for_plato(plato_id: int):
    """
    Carga el modelo correspondiente a un plato.
    ADAPTA esta función a tu estructura real:
    - Ejemplo actual: un XGBoost por plato guardado como `models/xgb_poisson/xgb_plato_<id>.json`
    - Si usas otro tipo (Random Forest, LSTM, etc.), ajusta aquí.
    """
    if plato_id <= 9:
        model_path = MODELS_DIR / "xgb_poisson_v2" / f"booster_poisson_plato_{plato_id}.json"
    else: # plato_id 10, 11, 12
        model_path = MODELS_DIR / "xgb_poisson_v2" / f"booster_reg_plato_{plato_id}.json"

    model = xgb.XGBRegressor()
    model.load_model(model_path)
    return model

# Ejemplo (si ya tienes modelos guardados):
# test_model = load_model_for_plato(1)
# test_model

## 7. Construcción del horizonte futuro a pronosticar


In [12]:
# Tomamos la última fecha disponible en el dataset histórico
last_date = df_hist[DATE_COL].max()
print("Última fecha histórica:", last_date.date())

# Horizonte futuro: próxima semana (o HORIZON_DAYS)
future_dates = pd.date_range(
    last_date + pd.Timedelta(days=1),
    periods=HORIZON_DAYS,
    freq="D"
)
future_dates



Última fecha histórica: 2025-12-31


DatetimeIndex(['2026-01-01', '2026-01-02', '2026-01-03', '2026-01-04',
               '2026-01-05', '2026-01-06', '2026-01-07'],
              dtype='datetime64[ns]', freq='D')

## 8. Forecast secuencial por plato

La lógica es la siguiente:

1. Tomar la serie histórica de un plato.
2. Para cada día futuro:
   - Agregar una nueva fila con esa fecha y `target` vacío.
   - Recalcular *features* (calendario + lags/rolling) sobre toda la serie (histórico + futuro).
   - Tomar los features del día en cuestión y pasarlos al modelo.
   - Guardar la predicción y considerarla como "venta simulada" para los siguientes días.

De esta forma, los lags (`lag_1`, `lag_7`, etc.) de días futuros usan tanto ventas reales
como predicciones previas.



In [13]:
df_hist.head()

,fecha,plato,cantidad,monto_total,anio,mes,dia,dow,fin_de_semana,feriado,...,dow_cos,mes_sin,mes_cos,year,weekofyear,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start
0,2021-01-15,1,13.0,338.0,2021,1,15,4,0,0,...,-0.900969,0.5,0.866025,2021,2,15,0,0,0,0
1,2021-01-16,1,27.0,702.0,2021,1,16,5,1,0,...,-0.222521,0.5,0.866025,2021,2,16,0,0,0,0
2,2021-01-17,1,19.0,494.0,2021,1,17,6,1,0,...,0.623490,0.5,0.866025,2021,2,17,0,0,0,0
3,2021-01-18,1,15.0,390.0,2021,1,18,0,0,0,...,1.000000,0.5,0.866025,2021,3,18,0,0,0,0
4,2021-01-19,1,17.0,442.0,2021,1,19,1,0,0,...,0.623490,0.5,0.866025,2021,3,19,0,0,0,0


# **XGBOOST**

In [14]:
all_forecasts = []

for plato_id, g in df_hist.groupby(PLATO_COL):
    g = g.sort_values(DATE_COL).copy()

    # 1) Verificar que el modelo exista en disco
    model_path_check = MODELS_DIR / "xgb_poisson_v2" / f"booster_poisson_plato_{plato_id}.json"
    if not model_path_check.exists():
        print(f"Error: Model file not found for plato_id {plato_id} at {model_path_check}")
        continue  # saltar plato si no hay modelo

    model = load_model_for_plato(int(plato_id))

    # 2) Usar EXACTAMENTE las features con las que se entrenó el modelo
    model_features = model.get_booster().feature_names

    # 3) Copia de trabajo del df del plato
    g_forecast = g.copy()

    for fd in future_dates:
        # 3.1 Agregar fila futura vacía
        new_row = {col: np.nan for col in g_forecast.columns}
        new_row[DATE_COL] = fd
        g_forecast = pd.concat([g_forecast, pd.DataFrame([new_row])], ignore_index=True)

        # 3.2 Recalcular calendario + lags/rolling
        g_forecast = add_calendar_features(g_forecast, DATE_COL)
        g_forecast = add_lags_rollings(g_forecast, TARGET_COL)

        # 3.3 Asegurar que TODAS las columnas esperadas existan
        for col in model_features:
            if col not in g_forecast.columns:
                g_forecast[col] = 0.0  # si no estaba, la creamos en 0

        # 3.4 Tomar la última fila de esa fecha
        row_fd = g_forecast[g_forecast[DATE_COL] == fd].sort_values(DATE_COL).tail(1)

        # 👉 AQUÍ LA CLAVE: usar model_features, NO FEATURE_COLS
        X_fd = row_fd[model_features]

        if X_fd.isna().any(axis=None):
            X_fd = X_fd.fillna(0.0)

        # 3.5 Predicción
        y_hat = model.predict(X_fd)[0]

        # 3.6 Guardar predicción como "venta" simulada
        g_forecast.loc[g_forecast[DATE_COL] == fd, TARGET_COL] = y_hat

        all_forecasts.append({
            "fecha": fd,
            "id_plato": int(plato_id),
            "demanda_predicha": float(y_hat)
        })

forecast_plato_xgboost = pd.DataFrame(all_forecasts)
forecast_plato_xgboost = forecast_plato_xgboost.sort_values(["fecha","id_plato"]).reset_index(drop=True)

print("Shape forecast_plato:", forecast_plato_xgboost.shape)
forecast_plato_xgboost.head()


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\40809135.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\40809135.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\40809135.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option(

Error: Model file not found for plato_id 10 at C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\models\xgb_poisson_v2\booster_poisson_plato_10.json
Error: Model file not found for plato_id 11 at C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\models\xgb_poisson_v2\booster_poisson_plato_11.json
Error: Model file not found for plato_id 12 at C:\Users\josep\OneDrive\Desktop\PUCP\2025-2\Tesis2\Resultados esperados\RESULTADO 3\Modelo\-1INF46-Plan_Compras_Produccion\models\xgb_poisson_v2\booster_poisson_plato_12.json
Shape forecast_plato: (63, 3)


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\40809135.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\40809135.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\40809135.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option(

,fecha,id_plato,demanda_predicha
0,2026-01-01,1,19.877472
1,2026-01-01,2,18.862059
2,2026-01-01,3,14.478246
3,2026-01-01,4,17.427208
4,2026-01-01,5,12.735395


# **RANDOM FOREST**

In [15]:
all_forecasts = []

for plato_id, g in df_hist.groupby(PLATO_COL):
    g = g.sort_values(DATE_COL).copy()

    # 1) Verificar que el modelo exista en disco
    model_path_check = MODELS_DIR / "Random_Forest_Final" / f"plato_{plato_id}.pkl"
    if not model_path_check.exists():
        print(f"Error: Model file not found for plato_id {plato_id} at {model_path_check}")
        continue  # saltar plato si no hay modelo

    model = load_model_for_plato(int(plato_id))

    # 2) Usar EXACTAMENTE las features con las que se entrenó el modelo
    model_features = model.get_booster().feature_names

    # 3) Copia de trabajo del df del plato
    g_forecast = g.copy()

    for fd in future_dates:
        # 3.1 Agregar fila futura vacía
        new_row = {col: np.nan for col in g_forecast.columns}
        new_row[DATE_COL] = fd
        g_forecast = pd.concat([g_forecast, pd.DataFrame([new_row])], ignore_index=True)

        # 3.2 Recalcular calendario + lags/rolling
        g_forecast = add_calendar_features(g_forecast, DATE_COL)
        g_forecast = add_lags_rollings(g_forecast, TARGET_COL)

        # 3.3 Asegurar que TODAS las columnas esperadas existan
        for col in model_features:
            if col not in g_forecast.columns:
                g_forecast[col] = 0.0  # si no estaba, la creamos en 0

        # 3.4 Tomar la última fila de esa fecha
        row_fd = g_forecast[g_forecast[DATE_COL] == fd].sort_values(DATE_COL).tail(1)

        # 👉 AQUÍ LA CLAVE: usar model_features, NO FEATURE_COLS
        X_fd = row_fd[model_features]

        if X_fd.isna().any(axis=None):
            X_fd = X_fd.fillna(0.0)

        # 3.5 Predicción
        y_hat = model.predict(X_fd)[0]

        # 3.6 Guardar predicción como "venta" simulada
        g_forecast.loc[g_forecast[DATE_COL] == fd, TARGET_COL] = y_hat

        all_forecasts.append({
            "fecha": fd,
            "id_plato": int(plato_id),
            "demanda_predicha": float(y_hat)
        })

forecast_plato_rf = pd.DataFrame(all_forecasts)
forecast_plato_rf = forecast_plato_rf.sort_values(["fecha","id_plato"]).reset_index(drop=True)

print("Shape forecast_plato:", forecast_plato_rf.shape)
forecast_plato_rf.head()


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3210009515.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3210009515.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3210009515.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_o

Shape forecast_plato: (84, 3)


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3210009515.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3210009515.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3210009515.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_o

,fecha,id_plato,demanda_predicha
0,2026-01-01,1,19.877472
1,2026-01-01,2,18.862059
2,2026-01-01,3,14.478246
3,2026-01-01,4,17.427208
4,2026-01-01,5,12.735395


# **LSTM**

In [16]:
all_forecasts = []

for plato_id, g in df_hist.groupby(PLATO_COL):
    g = g.sort_values(DATE_COL).copy()

    # 1) Verificar que el modelo exista en disco
    model_path_check = MODELS_DIR / "LSTM_Models" / f"plato_{plato_id}.keras"
    if not model_path_check.exists():
        print(f"Error: Model file not found for plato_id {plato_id} at {model_path_check}")
        continue  # saltar plato si no hay modelo

    model = load_model_for_plato(int(plato_id))

    # 2) Usar EXACTAMENTE las features con las que se entrenó el modelo
    model_features = model.get_booster().feature_names

    # 3) Copia de trabajo del df del plato
    g_forecast = g.copy()

    for fd in future_dates:
        # 3.1 Agregar fila futura vacía
        new_row = {col: np.nan for col in g_forecast.columns}
        new_row[DATE_COL] = fd
        g_forecast = pd.concat([g_forecast, pd.DataFrame([new_row])], ignore_index=True)

        # 3.2 Recalcular calendario + lags/rolling
        g_forecast = add_calendar_features(g_forecast, DATE_COL)
        g_forecast = add_lags_rollings(g_forecast, TARGET_COL)

        # 3.3 Asegurar que TODAS las columnas esperadas existan
        for col in model_features:
            if col not in g_forecast.columns:
                g_forecast[col] = 0.0  # si no estaba, la creamos en 0

        # 3.4 Tomar la última fila de esa fecha
        row_fd = g_forecast[g_forecast[DATE_COL] == fd].sort_values(DATE_COL).tail(1)

        # 👉 AQUÍ LA CLAVE: usar model_features, NO FEATURE_COLS
        X_fd = row_fd[model_features]

        if X_fd.isna().any(axis=None):
            X_fd = X_fd.fillna(0.0)

        # 3.5 Predicción
        y_hat = model.predict(X_fd)[0]

        # 3.6 Guardar predicción como "venta" simulada
        g_forecast.loc[g_forecast[DATE_COL] == fd, TARGET_COL] = y_hat

        all_forecasts.append({
            "fecha": fd,
            "id_plato": int(plato_id),
            "demanda_predicha": float(y_hat)
        })

forecast_plato_lstm = pd.DataFrame(all_forecasts)
forecast_plato_lstm = forecast_plato_lstm.sort_values(["fecha","id_plato"]).reset_index(drop=True)

print("Shape forecast_plato:", forecast_plato_lstm.shape)
forecast_plato_lstm.head()


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3999685124.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3999685124.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3999685124.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_o

Shape forecast_plato: (84, 3)


C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3999685124.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3999685124.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_fd = X_fd.fillna(0.0)
C:\Users\josep\AppData\Local\Temp\ipykernel_7248\3999685124.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_o

,fecha,id_plato,demanda_predicha
0,2026-01-01,1,19.877472
1,2026-01-01,2,18.862059
2,2026-01-01,3,14.478246
3,2026-01-01,4,17.427208
4,2026-01-01,5,12.735395


In [17]:
len(forecast_plato_lstm)

84

## 9. Exportar `forecast_plato.csv` para usar en planeación de compras


In [23]:
forecast_path = BASE_DIR / "Evidencia"/ "Modelo_Salidas" / "result_forecast_plato_rf.csv"
forecast_plato_rf.to_csv(forecast_path, index=False)
forecast_path



WindowsPath('C:/Users/josep/OneDrive/Desktop/PUCP/2025-2/Tesis2/Resultados esperados/RESULTADO 3/Modelo/-1INF46-Plan_Compras_Produccion/Evidencia/Modelo_Salidas/result_forecast_plato_rf.csv')

In [24]:
forecast_path = BASE_DIR / "Evidencia"/ "Modelo_Salidas" / "result_forecast_plato_xg.csv"
forecast_plato_xgboost.to_csv(forecast_path, index=False)
forecast_path



WindowsPath('C:/Users/josep/OneDrive/Desktop/PUCP/2025-2/Tesis2/Resultados esperados/RESULTADO 3/Modelo/-1INF46-Plan_Compras_Produccion/Evidencia/Modelo_Salidas/result_forecast_plato_xg.csv')

In [25]:
forecast_path = BASE_DIR / "Evidencia"/ "Modelo_Salidas" / "result_forecast_plato_lstm.csv"
forecast_plato_lstm.to_csv(forecast_path, index=False)
forecast_path



WindowsPath('C:/Users/josep/OneDrive/Desktop/PUCP/2025-2/Tesis2/Resultados esperados/RESULTADO 3/Modelo/-1INF46-Plan_Compras_Produccion/Evidencia/Modelo_Salidas/result_forecast_plato_lstm.csv')